In [227]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')

ticker = "^NSEI"
data1 = yf.download(ticker, start="2018-01-01", end="2024-01-01")
initial = data1.copy()


[*********************100%%**********************]  1 of 1 completed


In [228]:
def ema(df,period,column,name):
    df[name] = df[column].ewm(span=period,adjust=False).mean()

In [229]:
# (a) MACD
def macd(df):
    ema(df,12,"Close","EMA_12")
    ema(df,26,"Close","EMA_26")
    df["MACD"] = df["EMA_12"] - df["EMA_26"]
    ema(df,9,"MACD","SignalLine")

#Generate Signals
def signals_macd(df):
    df["Signals"] = [0]*len(df)
    for i in range(1,len(df)):
        if(df["MACD"].iloc[i-1] <= df["SignalLine"].iloc[i-1] and df["MACD"].iloc[i] >= df["SignalLine"].iloc[i]):
            df["Signals"].iloc[i] = -1
        elif(df["MACD"].iloc[i-1] >= df["SignalLine"].iloc[i-1] and df["MACD"].iloc[i] <= df["SignalLine"].iloc[i]):
            df["Signals"].iloc[i] = 1


In [230]:
# (b) Bollinger
def bollingerBand(df,period,multiplier):
    
    df["Middle_BB"] = df["Close"].rolling(20).mean()
    std = df["Close"].rolling(period).std()
    df["Upper_BB"] = df["Middle_BB"] + multiplier * std
    df["Lower_BB"] = df["Middle_BB"] - multiplier * std

#Generate signals
def signals_bollinger(df):
    df["Signals"] = [0]*len(df)
    df["Signals"][df["Close"] >= df["Upper_BB"]] = -1
    df["Signals"][df["Close"] <= df["Lower_BB"]] = 1
    curr = 0
    for i in range(len(df)):
        if(curr == 0):
            curr = df["Signals"].iloc[i]

        else:
            x = curr * df["Signals"].iloc[i]
            if( x == 1 ):
                df["Signals"].iloc[i] = 0
            elif(x == -1):
                curr = -curr

In [231]:
#Strategy (long trading only)
def Strategy1(df,stopLoss,takeProfit):
    for i in range(len(df)):
        if(df["Signals"].iloc[i] == -1):
            df["Signals"].iloc[i] = 0
            break;
        elif(df["Signals"].iloc[i] == 1):
            break;

    initialCap = 10000000
    finalCap = initialCap
    stocks = 0
    buyPrice = 0
    for i in range(len(df)):
        
        if (stocks == 0 and df["Signals"].iloc[i] == 1):
            stocks = finalCap // df["Close"].iloc[i]
            buyPrice = df["Close"].iloc[i]
        elif (stocks != 0 and df["Signals"].iloc[i] == -1):
            finalCap += stocks * ( df["Close"].iloc[i] - buyPrice)
            stocks = 0
        #CHECK STOPLOSS
        elif (stocks != 0 and df["Close"].iloc[i] <= buyPrice * (100 - stopLoss) / 100):
            finalCap += stocks * ( df["Close"].iloc[i] - buyPrice)
            stocks = 0
        #CHECK TAKEPROFIT
        elif (stocks != 0 and df["Close"].iloc[i] >= buyPrice * (100 + takeProfit) / 100):
            finalCap += stocks * ( df["Close"].iloc[i] - buyPrice)
            stocks = 0
    
    returns = (finalCap - initialCap)*100/initialCap
    print("Initial Capital =", initialCap)
    print("Final Capital =", finalCap)
    print("Strategy Return (%) =", returns)

In [232]:
#Strategy with trailing stop loss
#Define profit interval such that if profit % increases to that of interval then stop loss also increases to that interval
#Example let SL = 5% and TP = 20 and interval = 5%
#Then if profit percent reaches 5% then SL changes to 0%
#If profit percent reaches 10% then SL changes to -5%
def Strategy2(df,initialSL,TP,interval):
    for i in range(len(df)):
        if(df["Signals"].iloc[i] == -1):
            df["Signals"].iloc[i] = 0
            break;
        elif(df["Signals"].iloc[i] == 1):
            break;

    initialCap = 10000000
    finalCap = initialCap
    stocks = 0
    buyPrice = 0
    minusSL = 0
    SL = initialSL
    for i in range(len(df)):
        
        if (stocks == 0 and df["Signals"].iloc[i] == 1):
            stocks = finalCap // df["Close"].iloc[i]
            buyPrice = df["Close"].iloc[i]
        elif (stocks != 0 and df["Signals"].iloc[i] == -1):
            finalCap += stocks * ( df["Close"].iloc[i] - buyPrice)
            stocks = 0
        #CHECK STOPLOSS
        elif (stocks != 0 and df["Close"].iloc[i] <= buyPrice * (100 - SL) / 100):
            finalCap += stocks * ( df["Close"].iloc[i] - buyPrice)
            stocks = 0
        #CHECK TAKEPROFIT
        elif (stocks != 0 and df["Close"].iloc[i] >= buyPrice * (100 + TP) / 100):
            finalCap += stocks * ( df["Close"].iloc[i] - buyPrice)
            stocks = 0
        #CHECK INTERVAL
        elif(stocks != 0 and df["Close"].iloc[i] > buyPrice ):
            prof_per = ( df["Close"].iloc[i] - buyPrice ) * 100 / buyPrice
            if(prof_per >= (minusSL+1) * interval):
                minusSL = prof_per//interval
                SL = initialSL - minusSL*interval

    returns = (finalCap - initialCap)*100/initialCap
    print("Initial Capital =", initialCap)
    print("Final Capital =", finalCap)
    print("Strategy Return (%) =", returns)

In [233]:
#Denioising Algorithm- HeikinAshi
def HeikenAshi(df):
    data = pd.DataFrame()
    data["Close"] = (df["Open"] + df["Close"] + df["High"] +df["Low"])/4
    data["Open"] = float('NaN')
    for i in range(1,len(data)):
        data.iat[i,1] = (df.iat[i-1,0] + df.iat[i-1,3])/2
    data["High"] = np.maximum(df["High"], np.maximum(data["Open"], data["Close"]))
    data["Low"] = np.minimum(df["Low"], np.minimum(data["Open"], data["Close"]))
    return data

Without Denoising

In [234]:
#MACD
macd(data1)
signals_macd(data1)
data1

,Open,High,Low,Close,Adj Close,Volume,EMA_12,EMA_26,MACD,SignalLine,Signals
Date,,,,,,,,,,,
2018-01-02,10477.549805,10495.200195,10404.650391,10442.200195,10442.200195,153400,10442.200195,10442.200195,0.000000,0.000000,0
2018-01-03,10482.650391,10503.599609,10429.549805,10443.200195,10443.200195,167300,10442.354041,10442.274269,0.079772,0.015954,-1
2018-01-04,10469.400391,10513.000000,10441.450195,10504.799805,10504.799805,174900,10451.961082,10446.905791,5.055291,1.023822,0
2018-01-05,10534.250000,10566.099609,10520.099609,10558.849609,10558.849609,180900,10468.405471,10455.197925,13.207546,3.460567,0
2018-01-08,10591.700195,10631.200195,10588.549805,10623.599609,10623.599609,169000,10492.281492,10467.672124,24.609368,7.690327,0
...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,21295.849609,21390.500000,21232.449219,21349.400391,21349.400391,284700,21109.357460,20702.507591,406.849869,400.391457,0
2023-12-26,21365.199219,21477.150391,21329.449219,21441.349609,21441.349609,219500,21160.433176,20757.236629,403.196546,400.952475,0
2023-12-27,21497.650391,21675.750000,21495.800781,21654.750000,21654.750000,256500,21236.481918,20823.719101,412.762817,403.314543,0


In [235]:
#Strategy on MACD
print("From Strategy 1: ")
Strategy1(data1,5,10)
print("")
print("From Strategy 2: ")
Strategy2(data1,5,10,3)

From Strategy 1: 
Initial Capital = 10000000
Final Capital = 14004422.224609375
Strategy Return (%) = 40.04422224609375

From Strategy 2: 
Initial Capital = 10000000
Final Capital = 14699301.967773438
Strategy Return (%) = 46.993019677734374


In [236]:
#Bollingerband
data2 = initial.copy()
bollingerBand(data2,20,2)
signals_bollinger(data2)
data2

,Open,High,Low,Close,Adj Close,Volume,Middle_BB,Upper_BB,Lower_BB,Signals
Date,,,,,,,,,,
2018-01-02,10477.549805,10495.200195,10404.650391,10442.200195,10442.200195,153400,NaN,NaN,NaN,0
2018-01-03,10482.650391,10503.599609,10429.549805,10443.200195,10443.200195,167300,NaN,NaN,NaN,0
2018-01-04,10469.400391,10513.000000,10441.450195,10504.799805,10504.799805,174900,NaN,NaN,NaN,0
2018-01-05,10534.250000,10566.099609,10520.099609,10558.849609,10558.849609,180900,NaN,NaN,NaN,0
2018-01-08,10591.700195,10631.200195,10588.549805,10623.599609,10623.599609,169000,NaN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...
2023-12-22,21295.849609,21390.500000,21232.449219,21349.400391,21349.400391,284700,20831.387500,21877.731494,19785.043506,0
2023-12-26,21365.199219,21477.150391,21329.449219,21441.349609,21441.349609,219500,20913.720020,21872.033026,19955.407013,0
2023-12-27,21497.650391,21675.750000,21495.800781,21654.750000,21654.750000,256500,21001.972559,21885.382787,20118.562330,0


In [237]:
#Strategy on BollingerBand
print("From Strategy 1: ")
Strategy1(data2,5,10)
print("")
print("From Strategy 2: ")
Strategy2(data2,5,10,3)

From Strategy 1: 
Initial Capital = 10000000
Final Capital = 13026461.3359375
Strategy Return (%) = 30.264613359375

From Strategy 2: 
Initial Capital = 10000000
Final Capital = 10518677.3359375
Strategy Return (%) = 5.186773359375


With Denioising using HeikinAshi

In [238]:
heikin1 = HeikenAshi(initial)
initialHA = heikin1

In [239]:
#MACD
macd(heikin1)
signals_macd(heikin1)
#Strategy on MACD
print("From Strategy 1: ")
Strategy1(heikin1,5,10)
print("")
print("From Strategy 2: ")
Strategy2(heikin1,5,10,3)

From Strategy 1: 
Initial Capital = 10000000
Final Capital = 12792782.0859375
Strategy Return (%) = 27.927820859375

From Strategy 2: 
Initial Capital = 10000000
Final Capital = 11682432.84350586
Strategy Return (%) = 16.824328435058593


In [240]:
#Bollingerband
heikin2 = initialHA.copy()
bollingerBand(heikin2,20,2)
signals_bollinger(heikin2)
heikin2
#Strategy on BollingerBand
print("From Strategy 1: ")
Strategy1(heikin2,5,10)
print("")
print("From Strategy 2: ")
Strategy2(heikin2,5,10,3)

From Strategy 1: 
Initial Capital = 10000000
Final Capital = 10185686.59008789
Strategy Return (%) = 1.8568659008789064

From Strategy 2: 
Initial Capital = 10000000
Final Capital = 10522768.206054688
Strategy Return (%) = 5.227682060546875
